In [ ]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')



import pandas as pd
import numpy as np
import re

gdp_years = [str(x) for x in range(2006, 2016)]

def clean_names(x):
    nums = re.match('(\D*)\d*', x)
    if nums:
        x = nums.group(1)

    if x == "Republic of Korea": return "South Korea"
    if x == "United States of America": return "United States"
    if x == "United Kingdom of Great Britain and Northern Ireland": return "United Kingdom"
    if x == "China, Hong Kong Special Administrative Region": return "Hong Kong"
    
    pars = re.match('(.*)\(.*\)', x)
    if pars: return pars.group(1).strip()
    
    return x

Energy = pd.read_excel('assets/Energy Indicators.xls',
                           skiprows=17, 
                           skipfooter=38,
                           na_values=['...']).iloc[:, 2:]
Energy.rename({'Unnamed: 2': 'Country',
               'Petajoules': 'Energy Supply',
               'Gigajoules': 'Energy Supply per Capita',
               '%': '% Renewable'}, inplace=True, axis='columns')
Energy['Energy Supply'] = Energy['Energy Supply'].apply(lambda x: x * 1_000_000)
Energy['Country'] = Energy['Country'].apply(clean_names)


GDP = pd.read_csv('assets/world_bank.csv',  skiprows=4)
GDP['Country Name'].replace(["Korea, Rep.", "Iran, Islamic Rep.", "Hong Kong SAR, China"], 
                            ["South Korea", "Iran", "Hong Kong"],
                            inplace=True)
GDP.rename({'Country Name': 'Country'}, inplace=True, axis='columns')
GDP = GDP.filter(items=['Country'] + gdp_years)

ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')


def answer_one():  
    return ScimEn[:15].merge(Energy, how='inner', on='Country').merge(GDP, how='inner', on='Country').set_index('Country')


def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")